# メモ
- 2GPU以上での実行をお願いします。
- また途中でメモリ解放のためのカーネル再起動があります。
- GPTQモデルロード時に「CUDA extension not installed.」が表示されることがありますが、  
下記実行結果の通り、以降のセル実行に影響はありません。

## 環境設定
## ライブラリのインストール(omnicampus上の環境にないライブラリをインストールした方のみ)

In [1]:
import sys, os, warnings, gc, time
warnings.filterwarnings('ignore')

In [2]:
#演習環境
# GPUの確認、枚数に応じaて割り当てられた計算資源の消費速度が異なるのでお気をつけください。コンペ期間中は2時間で自動でインスタンスが落ちず、無制限となるため、インスタンスの停止し忘れにもご注意ください。
# 50GPU時間を超えると、インスタンスを立ち上げることができませんのでご注意ください。
!nvidia-smi

# /workspace/assets以下のファイルは永続化されます。作業中のcodeや学習した重みやデータはこちらに保存してください。(容量数TB、IOは遅い)
# /workspace/assets以外の部分のデータ容量は100GBまでです。複数モデルをダウンロードしている場合は容量にご注意ください。
# 具体的には以下のフォルダを適宜消す、あるいは/workspace/assets以下に移動するなどしてください。
os.environ['HF_HOME'] = '/workspace/hf'

!pip install -q "optimum>=1.12.0" openpyxl==3.1.2
!pip install -q auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/


Fri Oct 13 00:37:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   61C    P8              14W /  72W |     87MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## ファインチューニングモデルにおける学習データの利用および事後学習における学習データの利用可否をチェック

In [ ]:
#他者公開のモデルにおける違反データ利用有無のチェック
#1行でも同じデータがあった場合、違反データとしてFalseを設定し、対象データセット名を表示
from datasets import load_dataset

def confirm_data_usage(dataset):

    dataset = list(set(dataset))
    flag = True

    #jglue
    check_data="shunk031/JGLUE"
    name="MARC-ja"
    split="validation"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check["sentence"]))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    name="JCoLA"
    split="validation"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['sentence']+check['original']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    jcola = load_dataset(check_data, name=name, split=split)['original']

    split="validation_out_of_domain"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['sentence']+check['original']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    split="validation_out_of_domain_annotated"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['sentence']+check['original']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    name="JSTS"
    split="validation"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['sentence1']+check['sentence2']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    name="JNLI"
    split="validation"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['sentence1']+check['sentence2']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    name="JSQuAD"
    split="validation"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['context']+check['question']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    name="JCommonsenseQA"
    split="validation"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['question']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    #xlsum
    check_data="csebuetnlp/xlsum"
    name="japanese"
    split="validation"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['text']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    split="test"
    check = load_dataset(check_data, name=name, split=split)
    check = list(set(check['text']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    #elyza
    check_data="elyza/ELYZA-tasks-100"
    name=""
    split="test"
    check = load_dataset(check_data, split=split)
    check = list(set(check['input']+check['output']))
    if len(dataset+check) != len(set(dataset+check)):
        print("xxx NG xxx")
        print("     check_data:",check_data)
        print("     name:",name)
        print("     split:",split)
        flag = False

    if flag: print("ooo OK ooo")

In [ ]:
#重複した行を表示
def get_duplicate_row(train, test):
    dup = set(train) & set(test)
    print("--- train_row : text ---")
    print([str(i) + ":" +  text for i,text in enumerate(train) if text in dup])
    print("--- test_row : text ---")
    print([str(i) + ":" +  text for i,text in enumerate(test) if text in dup])

### [lightblue/openorca_stx](https://huggingface.co/lightblue/openorca_stx)におけるFT利用データを確認

In [ ]:
#SNOW T15：原文（日本語）をチェック
train_data = load_dataset("snow_simplified_japanese_corpus", name="snow_t15", split="train")
print("-"*20)
print("# snow_t15")
check = confirm_data_usage(train_data["original_ja"])

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

--------------------
# snow_t15


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

"None of [Index(['review_body', 'star_rating', 'review_id'], dtype='object')] are in the [columns]"


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#SNOW T23：原文（日本語）をチェック
train_data = load_dataset("snow_simplified_japanese_corpus", name="snow_t23", split="train")
print("-"*20)
print("# snow_t23")
check = confirm_data_usage(train_data["original_ja"])

Generating train split:   0%|          | 0/34300 [00:00<?, ? examples/s]

--------------------
# snow_t23
ooo OK ooo


In [ ]:
#TyDiQA (Ja)：質問文をチェック
train_data = load_dataset("khalidalt/tydiqa-goldp", name="japanese", split="train")
print("-"*20)
print("# tydiqa")
check = confirm_data_usage(train_data["question_text"])

ooo OK ooo


In [ ]:
#XLSUM (Ja)：テキスト（原文）をチェック
train_data = load_dataset("csebuetnlp/xlsum", "japanese", split="train")
print("-"*20)
print("# xlsum")
check = confirm_data_usage(train_data["text"])

--------------------
# xlsum
ooo OK ooo


In [ ]:
#snow_t15の重複チェック
train_data = load_dataset("snow_simplified_japanese_corpus", name="snow_t15", split="train")
test_data = load_dataset("shunk031/JGLUE", name="JCoLA", split="validation")
print("#"*20)
print("# validation")
get_duplicate_row(train_data["original_ja"], test_data["original"])

test_data = load_dataset("shunk031/JGLUE", name="JCoLA", split="validation_out_of_domain")
print("#"*20)
print("# validation_out_of_domain")
get_duplicate_row(train_data["original_ja"], test_data["original"])

test_data = load_dataset("shunk031/JGLUE", name="JCoLA", split="validation_out_of_domain_annotated")
print("#"*20)
print("# validation_out_of_domain_annotated")
get_duplicate_row(train_data["original_ja"], test_data["original"])

####################
# validation
--- train_row : text ---
['49586:今日は天気が良い。']
--- test_row : text ---
['330:今日は天気が良い。']
####################
# validation_out_of_domain
--- train_row : text ---
['49810:ドアが開いた。']
--- test_row : text ---
['422:ドアが開いた。']
####################
# validation_out_of_domain_annotated
--- train_row : text ---
['49810:ドアが開いた。']
--- test_row : text ---
['422:ドアが開いた。']


#### ※重複があったが、内容がよくある文章であり、タスクも違う文章のため、対象モデルは利用可能と判断
- SNOW：要約
- JCoLA：文書分類

### [TheBloke/StableBeluga2-70B-GPTQ](https://huggingface.co/lightblue/openorca_stx)におけるFT利用データを確認

#### ※[Orca-styleのデータセット](https://huggingface.co/stabilityai/StableBeluga2#training-dataset)は固有データかつ英語データのため、
#### 　チェックは割愛し、対象モデルは利用可能と判断

In [ ]:
import torch

del train_data
del test_data
del check
del confirm_data_usage
gc.collect()

torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Mon Oct  9 04:43:53 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   60C    P8              14W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## モデル（StableBeluga2-70B-GPTQ）ダウンロード・読込

In [3]:
# モデルのダウンロードと読み込み
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed, logging
import torch

logging.set_verbosity(logging.CRITICAL)

set_seed(42)

start_time = time.time()

model_name = "TheBloke/StableBeluga2-70B-GPTQ"

tokenizer_name = model_name

bnb_4bit_compute_dtype="bfloat16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

use_fast=True
revision="main"


tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=use_fast)

model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto',
    #quantization_config=bnb_config,
    trust_remote_code=True,
    use_safetensors=True,
    revision=revision,
    )

sec_load1 = time.time() - start_time
print("#"*30)
print('ロード時間')
print(f"{sec_load1:.2f}s")
print(f"{sec_load1/60:.2f}m")
print(f"{sec_load1/60/60:.2f}h")

[2023-10-13 00:37:57,268] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


CUDA extension not installed.


##############################
ロード時間
155.77s
2.60m
0.04h


In [4]:
!nvidia-smi

Fri Oct 13 00:40:19 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   59C    P0              30W /  72W |  16403MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
model.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 0,
 'model.layers.14': 0,
 'model.layers.15': 0,
 'model.layers.16': 0,
 'model.layers.17': 0,
 'model.layers.18': 0,
 'model.layers.19': 0,
 'model.layers.20': 0,
 'model.layers.21': 0,
 'model.layers.22': 0,
 'model.layers.23': 0,
 'model.layers.24': 0,
 'model.layers.25': 0,
 'model.layers.26': 0,
 'model.layers.27': 0,
 'model.layers.28': 0,
 'model.layers.29': 0,
 'model.layers.30': 0,
 'model.layers.31': 0,
 'model.layers.32': 0,
 'model.layers.33': 0,
 'model.layers.34': 0,
 'model.layers.35': 0,
 'model.layers.36': 0,
 'model.layers.37': 0,
 'model.layers.38': 1,
 'model.layers.39': 1,
 'model.layers.40': 1,
 'model.layers.41': 1,
 'model.layers.42'

In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192, padding_idx=0)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (rotary_emb): LlamaRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=8192, out_features=32000, bias=False)
)

## 推論用関数（生成タスク）

In [7]:
def generation(model, tokenizer, test_data, params):
# モデル・GPUメモリによって扱える最大トークン数が異なるので注意
# タスクによっては入力文が長いものがあります、どのように対処するかは各自で工夫いただければと思います。
# token長の2乗に計算量とメモリが必要になるので、提出用の推論処理のGPU資源確保の際にはご注意ください。
# max_length = 2048
#max_length = 256

    count = 0
    for data in test_data:
        count += 1

        if data['task_type'] == 'generation':
            print("id :",count-1)
            max_length = 1024+128

            text = f"""\nまず以下の文章の内容を読み取り、正しく理解してください。そして適切な日本語で簡潔に回答してください。
            \n\n{data['text']}
            \n回答:"""

            token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
            token_ids = token_ids[:, -(max_length):]
            token_ids_size = len(token_ids[0])
            start_time = time.time()

            with torch.no_grad():
                output_ids = model.generate(
                    inputs=token_ids.to(model.device),
                    max_new_tokens=256,
                    min_new_tokens=1,
                    do_sample=False,
                    top_k=50,
                    top_p=0.98,
                    num_beams=params["num_beams"],
                    early_stopping=True,
                    no_repeat_ngram_size=params["no_repeat_ngram_size"],
                    repetition_penalty=params["repetition_penalty"],
                    max_time=params["max_time"],
                )

            print(f"{time.time() - start_time:.2f}s")
            del token_ids
            torch.cuda.empty_cache()
            output = tokenizer.decode(output_ids.tolist()[0][token_ids_size:], skip_special_tokens=True)
            del output_ids
            torch.cuda.empty_cache()
            print(output)
            print("#"*20)
        else:
            output=""

        data['answer'] = output
        del output
        torch.cuda.empty_cache()

    return test_data

## 生成タスクの推論（Type3）

In [8]:
import time
import random
import json

torch.cuda.empty_cache()

set_seed(42)

start_time = time.time()


params = {
    "num_beams":3,
    "no_repeat_ngram_size":5,
    "repetition_penalty":1.1,
    "max_time":60*15,
}

# 推論を行うデータの確認、テストデータはコンペ期間中に追加される可能性があります、追加した際はslackにてアナウンスを行うのでご確認ください。
test_data = json.load(open('new_test.json', 'r'))

generation_data = generation(model, tokenizer, test_data, params)

sec_gene = time.time() - start_time
print("#"*30)
print('推論時間')
print(f"{sec_gene:.2f}s")
print(f"{sec_gene/60:.2f}m")
print(f"{sec_gene/60/60:.2f}h")

id : 70
330.25s
「凄まじい勢いで吹き荒れる風が」
####################
id : 71
688.47s

1. 目標を明確に定める
2. 仕事に興味を持つための新しいスキルを学ぶ
3. 同僚や上司とのコミュニケーションを強化する
4. 自分の仕事がどう社会に貢献しているかを理解する
5. バランスのとれた生活を送る
####################
id : 72
281.70s
「あなたが本当にやりたいことを見つけたら、私はどんなに応援してあげるつもりだわ。」
####################
id : 73
516.26s


古代エジプトを学ぶには、その歴史、文化、芸術、建築、哲学、政治などの側面を理解する必要があります。また、その時代の重要な人物や出来事についても調べるべきです。
####################
id : 74
109.45s

1. 南
2. 東
####################
id : 75
344.51s
私たちは全力を尽くして安全性を確保し、リソースを活用して環境負荷の軽揚を図ります。
####################
id : 76
903.78s
LLM (Language Learning with Machines) を使って英語を学ぶ初心者にとっての利点は、自分のペースで学習できる自律学習が可能であること、また自動評価システムによって正確さを向上させることが挙げられます。一方で、欠点としては、人間の先生が提供するような個人的なフィードバックが得られないことや、コミュニケーション能力の向上
####################
id : 77
441.16s
祖母に会えないことを残念に思っていること、近況を説明すること、祖母の健康状態について心配することなどを書くことができます。
####################
id : 78
904.14s


1. プレイヤが互いに協力して謎を解き、最終的にボスを倒すアドベンチャーゲーム。
2. 各プレイヤは異なる特殊能力を持ち、それらを組み合わせて課題を解決するパズルゲーム。 
3. オンラインでプレイできるコオペレーションマルチプレイヤRPG。
####################
id 

## 結果を保存

In [9]:
# 推論結果の保存、id, task_type, text, answerのkeyがあることを確認してください(正しく入っていない場合、スコア付けが行われません)
with open('submission.json', 'w') as f:
    json.dump(generation_data, f, indent=4)

check_submission_data = json.load(open('submission.json', 'r'))
print(len(check_submission_data))
print("-"*20)
print(check_submission_data[0])
print("-"*20)
print(check_submission_data[101])
print("-"*20)
print(check_submission_data[125])

90
--------------------
{'id': 0, 'task_type': 'multiple_choice', 'text': '物の外側をなす面を何と言うか？', 'choices': [{'choice_id': 1, 'text': '平面'}, {'choice_id': 2, 'text': '対面'}, {'choice_id': 3, 'text': '体'}, {'choice_id': 4, 'text': '表面'}, {'choice_id': 5, 'text': '顔面'}], 'answer': ''}
--------------------


IndexError: list index out of range

In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Oct 13 03:45:25 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 

## 推論用関数（選択タスク）

In [11]:
def choice(model, tokenizer, test_data):

    count = 0
    for data in test_data:
        count += 1

        if data['task_type'] == 'multiple_choice':
            print("id :",count-1)
            max_length = 1
            text =  f"あなたはクイズや常識問題が得意です。以下の例題のように適切な回答を出力してください。\n##########\n[問題]:日本の首都は？ \n[選択肢]:[1. ワシントン, 2. 東京, 3. ニューヨーク, 4. パリ, 5. 京都] \n[答えの選択肢番号]:2 \n（解説:この場合、日本の都道府県であり、かつ首都である選択肢は[2. 東京]だけなので、2が正解になります。）\n##########　\n[問題]:{data['text']}\n[選択肢]:[{data['choices'][0]['choice_id']}. {data['choices'][0]['text']}, {data['choices'][1]['choice_id']}. {data['choices'][1]['text']}, {data['choices'][2]['choice_id']}. {data['choices'][2]['text']}, {data['choices'][3]['choice_id']}. {data['choices'][3]['text']}, {data['choices'][4]['choice_id']}. {data['choices'][4]['text']}]\n[答えの選択肢番号]:"
            token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
            token_ids_size = len(token_ids[0])
            start_time = time.time()

            with torch.no_grad():
                output_ids = model.generate(
                    inputs=token_ids.to(model.device),
                    max_new_tokens=max_length,
                    do_sample=False,
                    min_new_tokens=max_length,
                )

            print(f"{time.time() - start_time:.2f}s")
            del token_ids
            torch.cuda.empty_cache()
            output = tokenizer.decode(output_ids.tolist()[0][token_ids_size:], skip_special_tokens=True)
            del output_ids
            torch.cuda.empty_cache()
            print(output)
            print("#"*20)
        else:
            output=""

        # 出力が1~5の数字になるようにする
        data['answer'] = next((int(char) for char in output if char in '12345'), random.randint(1, 5))
        del output
        torch.cuda.empty_cache()

    return test_data

## 選択タスクの推論（Type1）

In [12]:
import time
import random
import json

torch.cuda.empty_cache()

set_seed(42)

start_time = time.time()

# 推論を行うデータの確認、テストデータはコンペ期間中に追加される可能性があります、追加した際はslackにてアナウンスを行うのでご確認ください。
test_data = json.load(open('new_test.json', 'r'))

choice_data = choice(model, tokenizer, test_data)

sec_choi = time.time() - start_time
print("#"*30)
print('推論時間')
print(f"{sec_choi:.2f}s")
print(f"{sec_choi/60:.2f}m")
print(f"{sec_choi/60/60:.2f}h")

id : 0
7.07s
4
####################
id : 1
6.04s
1
####################
id : 2
6.06s
1
####################
id : 3
6.03s
3
####################
id : 4
6.01s
4
####################
id : 5
5.97s
5
####################
id : 6
5.94s
5
####################
id : 7
5.95s
5
####################
id : 8
5.91s
4
####################
id : 9
5.92s
1
####################
id : 10
5.95s
1
####################
id : 11
5.96s
3
####################
id : 12
5.95s
5
####################
id : 13
6.00s
5
####################
id : 14
6.01s
1
####################
id : 15
6.03s
3
####################
id : 16
6.01s
2
####################
id : 17
6.02s
4
####################
id : 18
6.00s
4
####################
id : 19
5.97s
4
####################
id : 20
5.97s
4
####################
id : 21
6.01s
5
####################
id : 22
5.93s
2
####################
id : 23
5.97s
1
####################
id : 24
6.00s
2
####################
id : 25
5.93s
5
####################
id : 26
5.92s
1
####################
id : 27
5.9

## 結果をマージ

In [13]:
submit_data = json.load(open('submission.json', 'r'))

for i in range(len(choice_data)):
    if submit_data[i]["task_type"] == "multiple_choice":
        submit_data[i]["answer"] = choice_data[i]["answer"]

# 推論結果の保存、id, task_type, text, answerのkeyがあることを確認してください(正しく入っていない場合、スコア付けが行われません)
with open('submission.json', 'w') as f:
    json.dump(submit_data, f, indent=4)

check_submission_data = json.load(open('submission.json', 'r'))
print(len(check_submission_data))
print("-"*20)
print(check_submission_data[0])
print("-"*20)
print(check_submission_data[101])
print("-"*20)
print(check_submission_data[125])

90
--------------------
{'id': 0, 'task_type': 'multiple_choice', 'text': '物の外側をなす面を何と言うか？', 'choices': [{'choice_id': 1, 'text': '平面'}, {'choice_id': 2, 'text': '対面'}, {'choice_id': 3, 'text': '体'}, {'choice_id': 4, 'text': '表面'}, {'choice_id': 5, 'text': '顔面'}], 'answer': 4}
--------------------


IndexError: list index out of range

In [14]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Oct 13 03:55:36 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 

# ！メモリ解放のためカーネル再起動をお願いします！

In [1]:
import sys, os, warnings, gc, time
warnings.filterwarnings('ignore')

In [2]:
#演習環境
# GPUの確認、枚数に応じaて割り当てられた計算資源の消費速度が異なるのでお気をつけください。コンペ期間中は2時間で自動でインスタンスが落ちず、無制限となるため、インスタンスの停止し忘れにもご注意ください。
# 50GPU時間を超えると、インスタンスを立ち上げることができませんのでご注意ください。
!nvidia-smi

# /workspace/assets以下のファイルは永続化されます。作業中のcodeや学習した重みやデータはこちらに保存してください。(容量数TB、IOは遅い)
# /workspace/assets以外の部分のデータ容量は100GBまでです。複数モデルをダウンロードしている場合は容量にご注意ください。
# 具体的には以下のフォルダを適宜消す、あるいは/workspace/assets以下に移動するなどしてください。
os.environ['HF_HOME'] = '/workspace/hf'

!pip install -q "optimum>=1.12.0" openpyxl==3.1.2
!pip install -q auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Fri Oct 13 03:56:50 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   70C    P8              20W /  72W |    123MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## モデル（openorca_stx）ダウンロード・読込

In [3]:
# モデルのダウンロードと読み込み
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed, logging
import torch

logging.set_verbosity(logging.CRITICAL)

set_seed(42)

start_time = time.time()

model_name = "lightblue/openorca_stx"

tokenizer_name = model_name

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

use_fast=False

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=use_fast)

model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True,
    #use_safetensors=True,
    #revision=revision,
    )

sec_load2 = time.time() - start_time
print("#"*30)
print('ロード時間')
print(f"{sec_load2:.2f}s")
print(f"{sec_load2/60:.2f}m")
print(f"{sec_load2/60/60:.2f}h")

[2023-10-13 03:57:16,738] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

##############################
ロード時間
113.17s
1.89m
0.03h


In [4]:
!nvidia-smi

Fri Oct 13 03:58:58 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   68C    P0              32W /  72W |   3453MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
model.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 0,
 'model.layers.14': 0,
 'model.layers.15': 0,
 'model.layers.16': 0,
 'model.layers.17': 1,
 'model.layers.18': 1,
 'model.layers.19': 1,
 'model.layers.20': 1,
 'model.layers.21': 1,
 'model.layers.22': 1,
 'model.layers.23': 1,
 'model.layers.24': 1,
 'model.layers.25': 1,
 'model.layers.26': 1,
 'model.layers.27': 1,
 'model.layers.28': 1,
 'model.layers.29': 1,
 'model.layers.30': 1,
 'model.layers.31': 1,
 'model.layers.32': 1,
 'model.layers.33': 1,
 'model.layers.34': 1,
 'model.layers.35': 1,
 'model.layers.36': 1,
 'model.layers.37': 1,
 'model.layers.38': 1,
 'model.layers.39': 1,
 'model.norm': 1,
 'lm_head': 1}

In [9]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32002, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


## 推論用関数（要約タスク）

In [10]:
def summarization(model, tokenizer, test_data):

    count = 0
    for data in test_data:
        count += 1

        if data['task_type'] == 'summarization':
            print("id :",count-1)
            max_length = 2048+256
            text = f"""{data['text']}\n\nあなたは要約のスペシャリストです。上記の日本語の文章を理解して日本語で要約してください。句点で文章が終わり、要約以外の文章（注釈や参照および備考）は不要です。\n\n要約:"""
            token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
            max_new_tokens = int(max(min(max_length, len(token_ids[0]))*0.1, 128))
            min_new_tokens = int(max(min(max_length, len(token_ids[0]))*0.025, 20))
            token_ids = token_ids[:, -(max_length):]
            token_ids_size = len(token_ids[0])
            start_time = time.time()

            with torch.no_grad():
                output_ids = model.generate(
                    inputs=token_ids.to(model.device),
                    do_sample=False,
                    max_new_tokens=max_new_tokens,
                    min_new_tokens=min_new_tokens,
                    no_repeat_ngram_size=10,
                    max_time=60*5,
                )

            print(f"{time.time() - start_time:.2f}s")
            del token_ids
            torch.cuda.empty_cache()
            output = tokenizer.decode(output_ids.tolist()[0][token_ids_size:], skip_special_tokens=True)
            del output_ids
            torch.cuda.empty_cache()
            print(output)
            print("#"*20)
        else:
            output=""

        data['answer'] = output
        del output
        torch.cuda.empty_cache()

    return test_data

## 要約タスクの推論(Type2)

In [11]:
import time
import random
import json

torch.cuda.empty_cache()

set_seed(42)

start_time = time.time()

# 推論を行うデータの確認、テストデータはコンペ期間中に追加される可能性があります、追加した際はslackにてアナウンスを行うのでご確認ください。
test_data = json.load(open('new_test.json', 'r'))

summarization_data = summarization(model, tokenizer, test_data)

sec_summ = time.time() - start_time
print("#"*30)
print('推論時間')
print(f"{sec_summ:.2f}s")
print(f"{sec_summ/60:.2f}m")
print(f"{sec_summ/60/60:.2f}h")

id : 50
256.80s
中国は13日、アメリカに対して対中関税を引き上げると発表した。アメリカが10日に中国からの輸入に対して25％の関税を追設したことを受けたもの。
####################
id : 51
233.36s
ネパールの世界遺産ルンビニは、大気汚染によって歴史的価値が傷つけられている。国際自然保護連合（IUCN）が調査した。
####################
id : 52
202.01s
英国の科学者たちは、英国の欧州連合（EU）離脱（ブレグジット）をめぐり、英国政府とEUとの間で科学分野における合意を求めている。
####################
id : 53
224.86s
米テキサス州で25日から続く熱帯低圧「ハービー」による大雨で、同州の州都ヒューストンでは28日までに2000人以上が救助された。
####################
id : 54
154.72s
スリランカで21日、イースター（復命祭）の日にカトリック教会やホステルなどが狙われ、少なくとも207人が死亡し、約500人が負傷した。
####################
id : 55
117.62s
米ニューヨーク市のマイケル・ブルームベルグ前市長が、2016年米大統領選に独立候補となる可能性を検討していたことが明らかになった。
####################
id : 56
164.21s
インドネシア・スマットラ島バタンガンサールで9月30日、村の警備員がニシキヘビと格戦し、勝利した。村の人たちはぬいぐるみのように巨大なヘビを吊るし、油で揚げ、みんなで食べた。
####################
id : 57
126.72s
ナイジェリア北部でイスラム教徒の武装組織「ボコ・ハラム」に拉致された少女たちの映像が公開さ され、政府は少女たちの解放に向かう努力を続けていると述べた。
####################
id : 58
175.43s
米大統領選の共和党候補指名をめぐり、実業家ドナルド・トランプ前大統領候補が26日、必要な代議哀数を獲得したと発表した。
####################
id : 59
242.54s
欧州連合（EU）は16日、トルコとの間で移民協定について合意し

## 結果をマージ

In [12]:
submit_data = json.load(open('submission.json', 'r'))

for i in range(len(summarization_data)):
    if submit_data[i]["task_type"] == "summarization":
        submit_data[i]["answer"] = summarization_data[i]["answer"]

# 推論結果の保存、id, task_type, text, answerのkeyがあることを確認してください(正しく入っていない場合、スコア付けが行われません)
with open('submission.json', 'w') as f:
    json.dump(submit_data, f, indent=4)

check_submission_data = json.load(open('submission.json', 'r'))
print(len(check_submission_data))
print("-"*20)
print(check_submission_data[0])
print("-"*20)
print(check_submission_data[101])
print("-"*20)
print(check_submission_data[125])

90
--------------------
{'id': 0, 'task_type': 'multiple_choice', 'text': '物の外側をなす面を何と言うか？', 'choices': [{'choice_id': 1, 'text': '平面'}, {'choice_id': 2, 'text': '対面'}, {'choice_id': 3, 'text': '体'}, {'choice_id': 4, 'text': '表面'}, {'choice_id': 5, 'text': '顔面'}], 'answer': 4}
--------------------


IndexError: list index out of range